In [ ]:
import logging
from kafka import KafkaProducer
import csv
from pyspark.sql import SparkSession
from kafka import KafkaConsumer
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType,TimestampType

from pyspark.sql.functions import col, split

In [ ]:
#Kafka Cinfig   
kafka_bootstrap_servers='master:9092'
topic_name = 'target-data-Sales'
# Create a Kafka consumer
consumer = KafkaConsumer(
    topic_name,
    bootstrap_servers=kafka_bootstrap_servers,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    value_deserializer=lambda x: x.decode('utf-8')  # Deserialize to string
)
df= pd.DataFrame()
print(df.shape[0])



0


In [4]:

# Create a Kafka producer
producer = KafkaProducer(bootstrap_servers=kafka_bootstrap_servers)

# Read CSV and send messages to Kafka
with open('target_data.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # Convert row to a string (or customize serialization)
        message = ','.join([str(row[key]) for key in row])

        # Send message to Kafka topic 
        producer.send(topic_name, key=None, value=message.encode('utf-8'))
print("data sent successfuly")
# Flush pending messages
producer.flush()

data sent successfuly


In [4]:

rows = []
for message in consumer:
    #print(f"Key: {message.key}, Value: {message.value}")
    row_data = [message.value]  # Assuming each message is a CSV row string
    #print(row_data)
    # Append to the list of rows
    rows.append(row_data)
    #print(len(rows))
    #  Limit processing for demonstration
    if len(rows) >= 1000:  # Process 100 messages, you can adjust this
        break

spark = SparkSession.builder \
    .appName("KafkaStreaming") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.2,org.apache.kafka:kafka-clients:3.2.2") \
    .enableHiveSupport()\
.config("spark.sql.warehouse.dir", "hdfs://namenode:8020/user/hive/warehouse") \

    .getOrCreate()        
df = spark.createDataFrame(rows,["csv_data"])


df.printSchema()
df = df.withColumn("id", split(col("csv_data"), ",")[0].cast("int")) \
       .withColumn("order_status", split(col("csv_data"), ",")[1]) \
       .withColumn("order_products_value", split(col("csv_data"), ",")[2].cast("double")) \
       .withColumn("order_freight_value", split(col("csv_data"), ",")[3].cast("double")) \
       .withColumn("order_items_qty", split(col("csv_data"), ",")[4].cast("int")) \
       .withColumn("order_purchase_timestamp", split(col("csv_data"), ",")[5]) \
       .withColumn("order_approved_at", split(col("csv_data"), ",")[6])

# Step 5: Check Schema
df.printSchema()

# Step 6: Remove Null Values
df = df.na.drop()

# Step 7: Exploratory Data Analysis (EDA)

# Calculate the mean of order_products_value and order_freight_value
df.select("order_products_value", "order_freight_value").show()

# Calculate mean of specific columns
df.agg({"order_products_value": "mean", "order_freight_value": "mean"}).show()

# Perform more EDA as needed (example: count, group by, etc.)
df.groupBy("order_status").count().show()
df.write.mode("overwrite").saveAsTable("Target_divya")

root
 |-- csv_data: string (nullable = true)

root
 |-- csv_data: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_products_value: double (nullable = true)
 |-- order_freight_value: double (nullable = true)
 |-- order_items_qty: integer (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)

+--------------------+-------------------+
|order_products_value|order_freight_value|
+--------------------+-------------------+
|                79.0|               17.8|
|               119.9|              27.16|
|              519.99|              41.69|
|                29.5|              17.92|
|               26.77|              23.11|
|               419.9|              23.02|
|                65.0|              16.21|
|               29.99|              19.82|
|               59.99|              51.14|
|               56.99|              16.13|
|             

In [5]:


df.write.mode("overwrite").saveAsTable("Target_divya")

2024-10-05 04:16:02,347 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2024-10-05 04:16:02,347 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
2024-10-05 04:16:08,579 WARN metastore.ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
2024-10-05 04:16:08,579 WARN metastore.ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore sclaersepb3lab0311@172.31.16.123
2024-10-05 04:16:08,598 WARN metastore.ObjectStore: Failed to get database default, returning NoSuchObjectException
2024-10-05 04:16:10,073 WARN scheduler.TaskSetManager: Lost task 0.0 in stage 26.0 (TID 412) (master.cloudloka.com executor 1): java.io.IOException: Mkdirs failed to create file:/home/sclaersepb3lab0311/spark-warehouse/target_divya/_temporary/0/_temporary/attempt_202410050416093688239167593542243_0026_

Py4JJavaError: An error occurred while calling o189.saveAsTable.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.datasources.DataSource.writeAndRead(DataSource.scala:550)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.saveDataIntoTable(createDataSourceTables.scala:220)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.run(createDataSourceTables.scala:177)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	a